In [1]:
%reset -f

In [9]:
#!/usr/bin/env python3
import pandas as pd
from typing import Union
import numpy as np
from tqdm import tqdm
import torch
from torch import nn
import re
import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = (15,10)

In [10]:
from scipy.signal import find_peaks

In [3]:
import bz2
import pickle
import _pickle as cPickle

In [4]:
# Load any compressed pickle file
def decompress_pickle(filename):
    data = bz2.BZ2File(filename, 'rb')
    data = cPickle.load(data)
    return data

In [7]:
# Pickle a file and then compress it into a file with extension 
def compressed_pickle(title, data):
    with bz2.BZ2File(title + '.pbz2', 'w') as f: 
        cPickle.dump(data, f)

In [8]:
def make_dataframe_from_data(data:dict)->pd.DataFrame:
    dataframe = pd.DataFrame(data=data)
    return dataframe

In [ ]:
def engineering_steps_v2(dataframe:pd.DataFrame)->pd.DataFrame:
    simulations = []
    thetas = []
    pitchs = []
    duty_cycles = []
    fill_factors = []
    lambdas = []
    trans = []

    count = 0
    num_rows = dataframe.shape[0]
    for i in tqdm(range(num_rows)):
        wavelengths = dataframe.iloc[i]['lambda']
        transmissions = dataframe.iloc[i]['trans']

        indices = find_peaks(transmissions, distance=150, prominence=0.15, width=0.2)[0]
        peaks_wavs = [wavelengths[j] for j in indices]
        peaks_trans = [transmissions[j] for j in indices]

        if len(peaks_wavs) > 0:
            check1 = max(peaks_wavs) > 1.35 and max(peaks_wavs) < 1.65
            check2 = check1 and max(peaks_trans) > 0.1
            # take out peaks that are less 0.1
            # take out peaks that lie outside 1.35 and 1.65
            check_wavelength = (max(peaks_wavs)) < 1.35 or (max(peaks_wavs) > 1.65)
            if len(indices) == 1 and check2:
                count = count + 1
                simulations.append(dataframe.iloc[i]['simulation'])
                thetas.append(dataframe.iloc[i]['theta'])
                pitchs.append(dataframe.iloc[i]['pitch'])
                duty_cycles.append(dataframe.iloc[i]['duty_cycle'])
                fill_factors.append(dataframe.iloc[i]['fill_factor'])
                lambdas.append(dataframe.iloc[i]['lambda'])
                trans.append(dataframe.iloc[i]['trans'])

    data = {'simulation':simulations, 'theta':thetas, 'pitch':pitchs, 'duty_cycle':duty_cycles, 'fill_factor':fill_factors, 'lambda':lambdas, 'trans':trans}
    return make_dataframe_from_data(data)

In [11]:
dataset_file = 'dataset/mydataset.pbz2'
dataframe = decompress_pickle(filename=dataset_file)
dataframe

,theta,pitch,duty_cycle,fill_factor,lambda,trans,simulation
0,5.0,5.000000e-07,0.4,0.200000,"[1.4, 1.40124, 1.4024899999999998, 1.40373, 1....","[0.00024694, 0.000222525, 0.000200177, 0.00017...",1
1,5.0,5.000000e-07,0.4,0.200000,"[1.4, 1.40124, 1.4024899999999998, 1.40373, 1....","[-0.000502307, -0.000521784, -0.000540792, -0....",2
2,5.0,5.000000e-07,0.4,0.257143,"[1.4, 1.40124, 1.4024899999999998, 1.40373, 1....","[0.000555697, 0.000525419, 0.000495302, 0.0004...",3
3,5.0,5.000000e-07,0.4,0.257143,"[1.4, 1.40124, 1.4024899999999998, 1.40373, 1....","[-0.000424704, -0.000424642, -0.000424468, -0....",4
4,5.0,5.000000e-07,0.4,0.280000,"[1.3, 1.3, 1.3, 1.3, 1.31, 1.31, 1.31, 1.31, 1...","[0.000155323, 0.000114162, 8.32e-05, 6.43e-05,...",5
...,...,...,...,...,...,...,...
11167,20.0,1.500000e-06,0.8,0.520000,"[1.3, 1.3, 1.3, 1.3, 1.31, 1.31, 1.31, 1.31, 1...","[-0.0180789, -0.0188972, -0.0199639, -0.021313...",11168
11168,20.0,1.500000e-06,0.8,0.542857,"[1.4, 1.40124, 1.4024899999999998, 1.40373, 1....","[0.00405141, 0.00416513, 0.00426947, 0.0043639...",11169
11169,20.0,1.500000e-06,0.8,0.542857,"[1.4, 1.40124, 1.4024899999999998, 1.40373, 1....","[-0.013056, -0.0132126, -0.0133639, -0.013507,...",11170
11170,20.0,1.500000e-06,0.8,0.600000,"[1.3, 1.3, 1.3, 1.3, 1.31, 1.31, 1.31, 1.31, 1...","[0.000377711, 0.0003538, 0.000336613, 0.000338...",11171


In [ ]:
example = 1
lambdas = np.flip(dataframe['lambda'][example])
plt.plot(lambdas, , label='FDTD')
plt.xlabel('lambda (micro-meter)', fontsize=16)
plt.ylabel('trans.', fontsize=16)
plt.legend(fontsize=16)